In [1]:
#Mount drive
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Move to dataset folder
#%cd /content/drive/MyDrive/Colab Notebooks/CS3244/Datasets/

In [3]:
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import bigrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lakbo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lakbo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
movie_details = pd.read_json('../Project/data/IMDB_movie_details.json', lines = True)
imdb_reviews = pd.read_json('../Project/data/IMDB_reviews.json', lines = True)

In [5]:
headers = movie_details.columns
headers

Index(['movie_id', 'plot_summary', 'duration', 'genre', 'rating',
       'release_date', 'plot_synopsis'],
      dtype='object')

In [6]:
no_summary = movie_details[movie_details['plot_summary'].isna()]
no_summary

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis


In [7]:
no_synopsis = movie_details[movie_details['plot_synopsis'].isna()]
no_synopsis

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis


In [8]:
imdb_reviews.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"


In [9]:
labels = imdb_reviews.is_spoiler
labels = labels.value_counts()
true_count = labels[True]
false_count = labels[False]
f'{true_count} + {false_count}'

'150924 + 422989'

In [10]:
def lowercase(dataframe,columnname):
    lowercase_dataframe = dataframe[columnname].apply(lambda x: x.lower())
    return lowercase_dataframe

def remove_special_characters(dataframe,columnname):
    dataframe_no_special_characters = dataframe[columnname].replace(r'[^A-Za-z0-9 ]+', '', regex=True)
    return dataframe_no_special_characters

def tokenize_words(dataframe,columnname):
    dataframe_tokenized_texts= dataframe[columnname].apply(lambda x: word_tokenize(x) )
    return dataframe_tokenized_texts

def remove_stop_words(dataframe,columnname):
    stop = stopwords.words('english')
    dataframe_no_stop_words= dataframe[columnname].apply(lambda x: [item for item in x if item not in stop])
    return dataframe_no_stop_words


In [11]:
#Processing of data
"""imdb_reviews = imdb_reviews.head()
imdb_reviews['review_text'] = lowercase(imdb_reviews,'review_text')
imdb_reviews['review_text'] = remove_special_characters(imdb_reviews,'review_text')
imdb_reviews['review_text'] = tokenize_words(imdb_reviews,'review_text')
imdb_reviews['review_text'] = remove_stop_words(imdb_reviews,'review_text')
imdb_reviews"""

"imdb_reviews = imdb_reviews.head()\nimdb_reviews['review_text'] = lowercase(imdb_reviews,'review_text')\nimdb_reviews['review_text'] = remove_special_characters(imdb_reviews,'review_text')\nimdb_reviews['review_text'] = tokenize_words(imdb_reviews,'review_text')\nimdb_reviews['review_text'] = remove_stop_words(imdb_reviews,'review_text')\nimdb_reviews"

In [12]:
#Processing of data
#change to lowercase
imdb_reviews['review_text'] = lowercase(imdb_reviews,'review_text')
#remove special characters
imdb_reviews['review_text'] = remove_special_characters(imdb_reviews,'review_text')
#tokenize text 15min
imdb_reviews['review_text'] = tokenize_words(imdb_reviews,'review_text')
#remove stop words 8min
imdb_reviews['review_text'] = remove_stop_words(imdb_reviews,'review_text')
imdb_reviews['is_spoiler'] = imdb_reviews['is_spoiler'].map({True: 1, False: 0})
imdb_reviews.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,1,"[oscar, year, shawshank, redemption, written, ...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,1,"[shawshank, redemption, without, doubt, one, b...",10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,1,"[believe, film, best, story, ever, told, film,...",8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,1,"[yes, spoilers, herethis, film, emotional, imp...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,1,"[heart, extraordinary, movie, brilliant, indel...",8,"Great story, wondrously told and acted"


In [13]:
#ngrams
imdb_reviews['bigrams'] = imdb_reviews['review_text'].apply(lambda x: list(bigrams(x)))

In [14]:
imdb_reviews['bigrams']

0         [(oscar, year), (year, shawshank), (shawshank,...
1         [(shawshank, redemption), (redemption, without...
2         [(believe, film), (film, best), (best, story),...
3         [(yes, spoilers), (spoilers, herethis), (heret...
4         [(heart, extraordinary), (extraordinary, movie...
                                ...                        
573908    [(go, wise), (wise, fast), (fast, pure), (pure...
573909    [(well, shall), (shall, say), (say, ones), (on...
573910    [(go, best), (best, movie), (movie, ever), (ev...
573911    [(call, 1999), (1999, teenage), (teenage, vers...
573912    [(movie, made), (made, doubt), (doubt, sucker)...
Name: bigrams, Length: 573913, dtype: object

In [15]:
imdb_reviews['bigram_text'] = imdb_reviews['bigrams'].apply(lambda x: ' '.join([' '.join(bigram) for bigram in x]))
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(imdb_reviews['bigram_text'])

In [22]:
temp = imdb_reviews[imdb_reviews['movie_id']=='tt0111161']
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(temp['bigram_text'])
temp

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary,bigrams,bigram_text
0,10 February 2006,tt0111161,ur1898687,1,"[oscar, year, shawshank, redemption, written, ...",10,A classic piece of unforgettable film-making.,"[(oscar, year), (year, shawshank), (shawshank,...",oscar year year shawshank shawshank redemption...
1,6 September 2000,tt0111161,ur0842118,1,"[shawshank, redemption, without, doubt, one, b...",10,Simply amazing. The best film of the 90's.,"[(shawshank, redemption), (redemption, without...",shawshank redemption redemption without withou...
2,3 August 2001,tt0111161,ur1285640,1,"[believe, film, best, story, ever, told, film,...",8,The best story ever told on film,"[(believe, film), (film, best), (best, story),...",believe film film best best story story ever e...
3,1 September 2002,tt0111161,ur1003471,1,"[yes, spoilers, herethis, film, emotional, imp...",10,Busy dying or busy living?,"[(yes, spoilers), (spoilers, herethis), (heret...",yes spoilers spoilers herethis herethis film f...
4,20 May 2004,tt0111161,ur0226855,1,"[heart, extraordinary, movie, brilliant, indel...",8,"Great story, wondrously told and acted","[(heart, extraordinary), (extraordinary, movie...",heart extraordinary extraordinary movie movie ...
...,...,...,...,...,...,...,...,...,...
4356,4 October 2000,tt0111161,ur0690751,0,"[movies, really, need, 2, hours, develop, stor...",7,"Unnecessarily long, but still excellent.","[(movies, really), (really, need), (need, 2), ...",movies really really need need 2 2 hours hours...
4357,4 August 2000,tt0111161,ur0819902,0,"[due, respect, people, complain, stark, good, ...",10,Quite possibly the best movie ever made,"[(due, respect), (respect, people), (people, c...",due respect respect people people complain com...
4358,26 July 2000,tt0111161,ur0562732,0,"[every, single, scene, shawshank, redemption, ...",9,Every once in a long while a truly inspiring f...,"[(every, single), (single, scene), (scene, sha...",every single single scene scene shawshank shaw...
4359,26 February 2004,tt0111161,ur2693433,0,"[beginning, thought, would, another, annoying,...",10,The second place (Top 250) is unfair for this ...,"[(beginning, thought), (thought, would), (woul...",beginning thought thought would would another ...


In [17]:
# Assuming the code for generating 'bigram_text' is executed
print(imdb_reviews['bigram_text'].head())


0    oscar year year shawshank shawshank redemption...
1    shawshank redemption redemption without withou...
2    believe film film best best story story ever e...
3    yes spoilers spoilers herethis herethis film f...
4    heart extraordinary extraordinary movie movie ...
Name: bigram_text, dtype: object


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# X: Features, y: Target variable
X = matrix # Assuming 'rating' and 'bigram_text' are features
y = temp['is_spoiler']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a logistic regression model
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 0.736540664375716
Confusion Matrix:
 [[575  88]
 [142  68]]
